<a href="https://colab.research.google.com/github/danielfrgs/2023_1-Aprendizaje_Maquina/blob/main/Practicas/P1/AM_P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 1: Análisis y Limpieza de datos

**Integrantes:**

> Cruz Larios Marcela

> Fragoso Alavardo Daniel

> Ayala Salcedo Marlene Viviane

> Gómez Martínez Jesús Enrique 

> Rangel Ramírez Hugo

# Librerias

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Se importan las librerías necesarias
import pandas as pd
import numpy as np
import random
import unicodedata
import re

import warnings
warnings.filterwarnings("ignore")

import datetime
from datetime import datetime
from datetime import date
import calendar
from IPython.display import HTML
from plotly.offline import plot,iplot
import matplotlib.pyplot as plt
import seaborn as sns
# pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go
import plotly.express as px #graficos express

In [3]:
#Gráficas

import cufflinks as cf
pd.set_option("display.max_columns",200)
pd.set_option('display.max_rows', 200)
warnings.filterwarnings("ignore")
cf.go_offline()
!pip install sklearn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import chisquare
from scipy import stats
from  functools import reduce
from scipy.stats import ksone
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=b86d79268ab6a6f5babf09c103558f3cd9f316f44dac99b25b3d1bfdc38e9fb5
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


# Funciones

## Calidad de Datos

In [4]:
#Se crea una función para verificar la completitud de las variables de la base.
def completitud(df):
    comple=pd.DataFrame(df.isnull().sum())
    comple.reset_index(inplace=True)
    comple=comple.rename(columns={"index":"columna",0:"total"})
    comple["completitud"]=(1-comple["total"]/df.shape[0])*100
    comple=comple.sort_values(by="completitud",ascending=True)
    comple.reset_index(drop=True,inplace=True)
    return comple

#Se crea una función en la cual se homologuen los simbolos de las variables de texto.
def clean_text(text, pattern="[^a-zA-Z0-9 ]"):
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("ascii"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().strip().split())
    return cleaned_text

## Gráficas

In [5]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
            },
          });
        </script>
        '''))

In [6]:
def cross_grafica(df,variable,variable_tgt):
    v_variables_new_by_loan_status = pd.crosstab(df[variable],df[variable_tgt])
    v_variables_new_by_loan_status_index = list(v_variables_new_by_loan_status.index)
    #valores
    malos = list(v_variables_new_by_loan_status.iloc[:,0])  
    buenos = list(v_variables_new_by_loan_status.iloc[:,1])

  #Configuraciones de la grafica
    plt.figure(figsize=(16,9))
  ## Se crean las primeras barras
    plt.bar(v_variables_new_by_loan_status_index, malos, label='0', color = "grey")
    
  ## Se crean las segundas barras y se apilan sobre las primeras
    plt.bar(v_variables_new_by_loan_status_index, buenos, label='1',  bottom=malos)
    plt.xticks(v_variables_new_by_loan_status_index, v_variables_new_by_loan_status_index) #indice y grupos en este caso es lo mismo (indice el numero de grupos y grupos las etiquetas)
    plt.ylabel("Frecuency")
    plt.xlabel(variable)
    plt.title(variable + '_' + variable_tgt)
    plt.legend()
    # plt.savefig(v_variables_new[i]+".png")
    plt.show()

## Missings

In [7]:
def T_E(train, test, columns):
    
    target = list(train.filter(like = 'tgt').columns)[0]
    
    variables_disc = []
    variables_v = []
    variables_c = []

    dic = {}
    iv = []    

    for col in columns:
        if col[0] == 'c':
            variables_c.append(col)
            variables_v.append('v'+col[1:])
            variables_disc.append(col[2:])
        elif col[0] == 'v':
            variables_v.append(col)
            
    for col in variables_c:
        i = 1
        suma = 0
        while i-1 == suma:
            suma = sum(pd.cut(train[col], bins = i).value_counts(1) >= 0.05)
            i += 1  

        train['v' + col[1:]] = pd.cut(train[col], bins = i-1).astype(str)
        
        if col[2:] in variables_disc:
            continue

    # Missings
    for col in variables_v:
        p = pd.DataFrame()
        train[col] = train[col].astype(str).replace({"nan":"Missings"})
        
        # WOE
        aux = train[[col, target]].groupby(col).agg(["count","sum"])
        aux["no_evento"] = aux[target,"count"]-aux[target,"sum"]
        aux["evento"] = aux[target,"sum"]
        aux["evento%"] = aux["evento"]/aux["evento"].sum()
        aux["no_evento%"] = aux["no_evento"]/aux["no_evento"].sum()
        aux["WOE"]=np.log(aux["evento%"]/aux["no_evento%"])
        aux["IV"]=(aux["evento%"]-aux["no_evento%"])*aux["WOE"]
        iv.append(round(aux["IV"].sum(), 5))
        aux = aux[["WOE"]].reset_index().rename(columns={"WOE":"W"+col[1:]})
        train = train.merge(aux,on=col, how="left")
        
        dic[col] = dict(zip(aux[col], aux["W"+col[1:]]))
       
        if col[2:] in variables_disc:
            for i in range(aux[col].shape[0]):
                intervalo = aux.iloc[i,0]
                
                if intervalo == 'Missings':
                    
                    test["W"+col[1:]].replace(to_replace = ['nan'], value = aux.iloc[i,1], inplace = True)
                    
                    
                else:
                    
                    inf = float(intervalo.split(',')[0][1:])
                    sup = float(intervalo.split(',')[1].strip()[0:-1])
                    
                    test[col] = np.where(test['c'+col[1:]].astype(float).between(inf, sup), intervalo, 'Missings')
                    test["W"+col[1:]] = np.where(test['c'+col[1:]].between(inf, sup), aux.iloc[i,1], 'nan')
                            
        else:
            test[col] = test[col].astype(str).replace({"nan":"Missings"})
            test = test.merge(aux,on=col, how="left")
    
    return train, test, dic, pd.DataFrame(zip(variables_v, iv), columns = ['Variables', 'IV'])

## Outliers

In [8]:
def imput_moda(col):
  X_train[col].value_counts()
  mode=X_train[col].mode()[0]
#Frecuencias observadas
#Valores
  x_i=X_train[col].fillna(mode).value_counts()
  #comprobación con valos crítico de la distribución
  k=x_i.sum()
  p_i=X_train[col].dropna().value_counts(1)
#Frecuencias teóricas esperadas
  m_i=k*p_i
  chi=chisquare(f_obs=x_i,f_exp=m_i)
  p_val=chi.pvalue
  alpha=0.05
  if p_val<alpha:
      print("Rechazamos HO(La porporción de categorias es la misma que la general)")
  else:
      print("Aceptamos HO(La porporción de categorias es la misma que la general)")
  return df[col].replace(np.nan,mode)

def c_imput(col):
   mean=X_train[col].mean()
   median=X_train[col].median()
   mode=X_train[col].mode()[0]   

   originales_mean=list(df[col].dropna().values)
   imputados_mean=list(df[col].fillna(mean).values)
   s_1=stats.ks_2samp(originales_mean,imputados_mean)
   statistic_1=s_1[0]
   pvalue_1=s_1[1]

   originales_median=list(df[col].dropna().values)
   imputados_median=list(df[col].fillna(median).values)
   s_2=stats.ks_2samp(originales_median,imputados_median)
   statistic_2=s_2[0]
   pvalue_2=s_2[1]

   
   originales_mode=list(df[col].dropna().values)
   imputados_mode=list(df[col].fillna(mode).values)
   s_3=stats.ks_2samp(originales_mode,imputados_mode)
   statistic_3=s_3[0]
   pvalue_3=s_3[1]
   
   ## metodo| p-value | estadistico
   # initialize list of lists
   datos = [['mean', pvalue_1,statistic_1], ['median', pvalue_2,statistic_2], ['mode', pvalue_3,statistic_3]]
 
   # Create the pandas DataFrame
   final_tableu = pd.DataFrame(datos, columns = ['Method', 'p-value','statistic'])

   how=final_tableu.iloc[final_tableu['p-value'].idxmax()]['Method']
   imp=SimpleImputer(missing_values=np.nan,strategy=how)
   imp.fit(X_train[[col]])
   X_train[[col]]=imp.transform(X_train[[col]])
   X_test[[col]]=imp.transform(X_test[[col]])


   print(how)
   print(final_tableu)

In [9]:
#multicolineaidad continuas
def calc_vif(X):
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return(vif)

#Multicolinealidad Categoricas
def multicolinealidad_cate(df,y,threshold=5,only_final_vif=False):
    '''
    Función para calcular el VIF Genralizado, (GVIF, Fox and Monette 1992)
    Las variables no deben de deben de estar transformadas con OneHotEncoder ya que la función lo realiza internamnete.
    Args:
        df
        y
        threshold
        only_final_vif
    
    Returns:
        pandas data frame: a data frame, indexed by factor of the GVIF, GVIF^(1/2Df), VIF^(1/2Df)^2 
    '''

    df_x = df.drop(y,axis=1)
    #Guardamos los nombres de las columnas añadiendo el prefijo

    onehot_list = list(df_x.select_dtypes(include=['category', 'object', 'string']).columns)
    #Generamos las variables dummy 
    
    df_1hot = pd.get_dummies(df_x, drop_first=True, dummy_na=False, prefix_sep='_')


    #Dataframe vacio para guardar los resultados
    gvif_df = pd.DataFrame(columns = ['factor','GVIF','Df','GVIF^(1/2Df)', 'VIF'])

    # Iteramos sobre las columas
    for (columnName, columnData) in df_x.iteritems():
        #Si se generaron las de una columna, es decir la variable tiene más de dos posibilidades utilizamos todas las variables creadas
        
        if columnName in onehot_list:
            X1 = df_1hot.loc[:, df_1hot.columns.str.startswith(columnName)]
            X2 = df_1hot.loc[:, ~df_1hot.columns.str.startswith(columnName)]
        else:
            X1 = df_1hot[[columnName]].values
            X2 = df_1hot.loc[:, df_1hot.columns != columnName].values
        #display(X1)
        # Calculamose gvif
        #A la matriz de correlación de las variables codificadas en caliente del atributo en consideración.
        #B la matriz de correlación de todos los demás atributos del conjunto de datos (uno codificado en dummy y numérico) excluyendo los de A.
        #C la matriz de correlación de las variables consideradas tanto en A como en B.
        #GVIF= (det(A)*det(B))/det(C)
        #display(np.corrcoef(X1, rowvar=False))
        gvif = np.linalg.det(np.array(np.corrcoef(X1, rowvar=False), ndmin=2)) * np.linalg.det(np.corrcoef(X2, rowvar=False)) / np.linalg.det(np.corrcoef(np.append(X1, X2, axis=1), rowvar=False))
        #(GVIF) elevado a (1 / (2 * grados de libertad))
        #Los grados de libertad es 1-n (n=Número de opciones que tiene la variable categorica)
        gvif_12df = np.power(gvif, 1 / (2 * X1.shape[1]))
        gvif_12df_sq = gvif_12df ** 2
        DF =  X1.shape[1]
    

        # 
        new_row = {'factor': columnName, 'GVIF': gvif,'Df':DF, 'GVIF^(1/2Df)': gvif_12df, 'VIF': gvif_12df_sq}
        gvif_df = gvif_df.append(new_row, ignore_index=True)

    gvif_df = gvif_df.set_index('factor')
    if only_final_vif:
        gvif_df_final = gvif_df.drop(['GVIF','Df','GVIF^(1/2Df)'],axis=1)
    else:
        gvif_df_final = gvif_df 
    gvif_filter = gvif_df.loc[gvif_df['VIF'] >= threshold]['VIF'].to_dict()
    if gvif_filter:
        for i in gvif_filter.keys():
            df_x_m = df_x.drop([i],axis=1)
    else:
        df_x_m = df_x
    df_m=pd.concat([df_x_m,df[y]],axis=1)
    return gvif_df_final

# Carga de Datos

### Sobre los Datos

Contamos con información de la empresa Lending Club, la cual es una empresa
estadounidense de préstamos, con sede en San Francisco, California. En esta empresa
los prestatarios pueden acceder fácilmente a préstamos con tasas de interés bajas.
El dataset contiene datos de todos los préstamos emitidos durante el período 2007-2015,
donde contiene el estado actual del préstamo (actual, cancelado, completamente pagado)
y la información de pago más reciente. Las características adicionales incluyen puntajes
de crédito, número de consultas financieras y cobros, entre otros.



In [10]:
df = pd.read_csv("/content/drive/MyDrive/Universidad/Ciencia de Datos/7mo Semestre/2023_1-Aprendizaje_Maquina/Practicas/P1/dataset_practica1.csv", encoding="utf-8")

# Calidad de Datos

In [11]:
# Dimensión de nuestro data frame
df.shape

(887379, 74)

In [12]:
#Se muestran los primeros 3 registros
df.head(3)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,NaN,NaN,3.0,0.0,13648.0,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.0,0.00,0.00,Jan-2015,171.62,NaN,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,NaN,NaN,3.0,0.0,1687.0,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.0,117.08,1.11,Apr-2013,119.66,NaN,Sep-2013,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,NaN,NaN,2.0,0.0,2956.0,98.5,10.0,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.0,0.00,0.00,Jun-2014,649.91,NaN,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Tipos de Variables:
#(Recordemos al final debemos de obtener un DataFrame con variables númericas únicamente)

df.dtypes

id                               int64
member_id                        int64
loan_amnt                      float64
funded_amnt                    float64
funded_amnt_inv                float64
term                            object
int_rate                       float64
installment                    float64
grade                           object
sub_grade                       object
emp_title                       object
emp_length                      object
home_ownership                  object
annual_inc                     float64
verification_status             object
issue_d                         object
loan_status                     object
pymnt_plan                      object
url                             object
desc                            object
purpose                         object
title                           object
zip_code                        object
addr_state                      object
dti                            float64
delinq_2yrs              

## Completitud

In [14]:
#Primero, se revisa la completitud de la Base antes de re-etiquetar las variables para así trabajar con menos datos, 
#donde solo ocuparemos las que cumplan con el 80% de la misma.
completitud_ = completitud(df)
colum_elegidas = completitud_[completitud_['completitud' ]>70]['columna'].values
colum_elegidas

array(['next_pymnt_d', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
       'emp_title', 'emp_length', 'last_pymnt_d', 'revol_util', 'title',
       'collections_12_mths_ex_med', 'last_credit_pull_d', 'open_acc',
       'pub_rec', 'earliest_cr_line', 'total_acc', 'delinq_2yrs',
       'acc_now_delinq', 'inq_last_6mths', 'annual_inc',
       'application_type', 'policy_code', 'last_pymnt_amnt',
       'collection_recovery_fee', 'id', 'total_rec_late_fee', 'member_id',
       'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'home_ownership',
       'verification_status', 'issue_d', 'recoveries', 'loan_status',
       'url', 'purpose', 'zip_code', 'addr_state', 'dti', 'revol_bal',
       'initial_list_status', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'pymnt_plan', 'out_prncp'], dtype=object)

In [15]:
len(colum_elegidas)

53

In [16]:
df_completitud = df[colum_elegidas]

## Etiquetado de Variables

In [17]:
colum_elegidas

# v = Numéricas: Discretas y continuas
# c = Categóricas
# d = Fechas
# t = Texto
# i = No sirve para el modelo

v = ['tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim', 'revol_util', 
       'collections_12_mths_ex_med',  'open_acc', 'pub_rec', 'total_acc', 'delinq_2yrs',
       'acc_now_delinq', 'inq_last_6mths', 'annual_inc', 'last_pymnt_amnt',
       'collection_recovery_fee', 'total_rec_late_fee',  'loan_amnt',
       'funded_amnt', 'funded_amnt_inv', 'int_rate',
       'installment', 'recoveries', 'dti', 'revol_bal',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'out_prncp']
c = ['last_pymnt_d', 'emp_length', 'policy_code', 'term', 'application_type',
     'grade', 'sub_grade', 'home_ownership', 'loan_status', 'purpose', 'zip_code',
     'addr_state', 'initial_list_status', 'pymnt_plan']
d = ['next_pymnt_d', 'last_credit_pull_d', 'earliest_cr_line',
     'verification_status', 'issue_d']
t = ['title', 'emp_title']
i = ['id', 'member_id', 'url']

new_names_v = {old_name:'v_'+ old_name for old_name in v }
new_names_c = {old_name:'c_'+ old_name for old_name in c }
new_names_t = {old_name:'t_'+ old_name for old_name in t }
new_names_d = {old_name:'d_'+ old_name for old_name in d }

df_etiquetado = df_completitud.rename(columns = new_names_v).rename(columns = new_names_c).rename(columns = new_names_d).rename(columns = new_names_t)

In [18]:
df_etiquetado.head()

,d_next_pymnt_d,v_tot_coll_amt,v_tot_cur_bal,v_total_rev_hi_lim,t_emp_title,c_emp_length,c_last_pymnt_d,v_revol_util,t_title,v_collections_12_mths_ex_med,d_last_credit_pull_d,v_open_acc,v_pub_rec,d_earliest_cr_line,v_total_acc,v_delinq_2yrs,v_acc_now_delinq,v_inq_last_6mths,v_annual_inc,c_application_type,c_policy_code,v_last_pymnt_amnt,v_collection_recovery_fee,id,v_total_rec_late_fee,member_id,v_loan_amnt,v_funded_amnt,v_funded_amnt_inv,c_term,v_int_rate,v_installment,c_grade,c_sub_grade,c_home_ownership,d_verification_status,d_issue_d,v_recoveries,c_loan_status,url,c_purpose,c_zip_code,c_addr_state,v_dti,v_revol_bal,c_initial_list_status,v_out_prncp_inv,v_total_pymnt,v_total_pymnt_inv,v_total_rec_prncp,v_total_rec_int,c_pymnt_plan,v_out_prncp
0,NaN,NaN,NaN,NaN,NaN,10+ years,Jan-2015,83.7,Computer,0.0,Jan-2016,3.0,0.0,Jan-1985,9.0,0.0,0.0,1.0,24000.0,INDIVIDUAL,1.0,171.62,0.00,1077501,0.00,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,RENT,Verified,Dec-2011,0.00,Fully Paid,https://www.lendingclub.com/browse/loanDetail....,credit_card,860xx,AZ,27.65,13648.0,f,0.0,5861.071414,5831.78,5000.00,861.07,n,0.0
1,NaN,NaN,NaN,NaN,Ryder,< 1 year,Apr-2013,9.4,bike,0.0,Sep-2013,3.0,0.0,Apr-1999,4.0,0.0,0.0,5.0,30000.0,INDIVIDUAL,1.0,119.66,1.11,1077430,0.00,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,RENT,Source Verified,Dec-2011,117.08,Charged Off,https://www.lendingclub.com/browse/loanDetail....,car,309xx,GA,1.00,1687.0,f,0.0,1008.710000,1008.71,456.46,435.17,n,0.0
2,NaN,NaN,NaN,NaN,NaN,10+ years,Jun-2014,98.5,real estate business,0.0,Jan-2016,2.0,0.0,Nov-2001,10.0,0.0,0.0,2.0,12252.0,INDIVIDUAL,1.0,649.91,0.00,1077175,0.00,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,RENT,Not Verified,Dec-2011,0.00,Fully Paid,https://www.lendingclub.com/browse/loanDetail....,small_business,606xx,IL,8.72,2956.0,f,0.0,3003.653644,3003.65,2400.00,603.65,n,0.0
3,NaN,NaN,NaN,NaN,AIR RESOURCES BOARD,10+ years,Jan-2015,21.0,personel,0.0,Jan-2015,10.0,0.0,Feb-1996,37.0,0.0,0.0,1.0,49200.0,INDIVIDUAL,1.0,357.48,0.00,1076863,16.97,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,RENT,Source Verified,Dec-2011,0.00,Fully Paid,https://www.lendingclub.com/browse/loanDetail....,other,917xx,CA,20.00,5598.0,f,0.0,12226.302212,12226.30,10000.00,2209.33,n,0.0
4,Feb-2016,NaN,NaN,NaN,University Medical Group,1 year,Jan-2016,53.9,Personal,0.0,Jan-2016,15.0,0.0,Jan-1996,38.0,0.0,0.0,0.0,80000.0,INDIVIDUAL,1.0,67.79,0.00,1075358,0.00,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,RENT,Source Verified,Dec-2011,0.00,Current,https://www.lendingclub.com/browse/loanDetail....,other,972xx,OR,17.94,27783.0,f,766.9,3242.170000,3242.17,2233.10,1009.07,n,766.9


## Limpieza

## Duplicados

## Consistencia

## Orden

## Normalización

## Precisión

# Outliers